In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import src.draw_graphs as dg

In [ ]:
Gs, Gp, Gn, G = dg.create_graph("Friends.csv")
dg.draw_strict_graph(Gs)

In [ ]:
dg.draw_graph(G, coloring=np.array([2, 1, 1, 2, 2, 1]))

In [ ]:
Hs, Hp, Hn, H = dg.create_graph("Wedding-seating-plan.csv")
dg.draw_graph(H, coloring=np.array([3, 3, 1, 3, 2, 1, 1, 2, 2, 3, 2, 2, 3, 3, 1]))

In [ ]:
dg.draw_graph(Hp, coloring=np.array([3, 3, 1, 3, 2, 1, 1, 2, 2, 3, 2, 2, 3, 3, 1]))

In [ ]:
dg.draw_graph(Hn, coloring=np.array([3, 3, 1, 3, 2, 1, 1, 2, 2, 3, 2, 2, 3, 3, 1]))